In [23]:
import os
import sys
import json
import numpy as np
import torch
from torch import nn
from torch import optim
from torch.optim import lr_scheduler

from opts import parse_opts
from model import generate_model
from mean import get_mean, get_std
from spatial_transforms import (
    Compose, Normalize, Scale, CenterCrop, CornerCrop, MultiScaleCornerCrop,
    MultiScaleRandomCrop, RandomHorizontalFlip, ToTensor)
from temporal_transforms import LoopPadding, TemporalRandomCrop
from target_transforms import ClassLabel, VideoID
from target_transforms import Compose as TargetCompose
from dataset import get_training_set, get_validation_set, get_test_set
from utils import Logger
from train import train_epoch
from validation import val_epoch
import test

import easydict

In [34]:
pwd

'/home/qq/CSCE689/3D-ResNets-PyTorch'

In [24]:

# opt = parse_opts()

# import argparse
# parser = argparse.ArgumentParser()
# parser.add_argument(
#         '--root_path',
#         default='/root/data/ActivityNet',
#         type=str,
#         help='Root directory path of data')
# parser.add_argument(
#         '--video_path',
#         default='video_kinetics_jpg',
#         type=str,
#         help='Directory path of Videos')


In [37]:

opt = easydict.EasyDict({
    "root_path": '',
    "video_path": 'video_kinetics_jpg',
    "annotation_path": 'kinetics.json',
    "result_path": 'results',
    "dataset": 'ucf101',
    "n_classes": 101,
    "n_finetune_classes": 101,
    "sample_size": 112,
    "sample_duration": 16,
    "initial_scale": 1.0,
    "n_scales": 5,
    "scale_step": 0.84089641525,
    "train_crop": 'corner',
    "learning_rate": 0.1,
    "momentum": 0.9,
    "dampening": 0.9,
    "weight_decay": 0.001,
    "mean_dataset": 'activitynet',
    "no_mean_norm": False,
    "std_norm": False,
    "nesterov": False,
    "optimizer": 'sgd',
    "lr_patience": 10,
    "batch_size": 128,
    "n_epochs": 200,
    "begin_epoch": 1,
    "n_val_samples": 3,
    "resume_path": '',
    "pretrain_path": '',
    "ft_begin_index": 0,
    "no_train": False,
    "no_val": False,
    "test": False,
    "test_subset": 'val',
    "scale_in_test": 1.0,
    "crop_position_in_test": 'c',
    "no_softmax_in_test": False,
    "no_cuda": False,
    "n_threads": 4,
    "checkpoint": 10,
    "no_hflip": False,
    "norm_value": 1,
    "model": 'resnet',
    "model_depth": 18,
    "resnet_shortcut": 'B',
    "wide_resnet_k": 2,
    "resnext_cardinality": 32,
    "manual_seed": 1
})

In [38]:

if opt.root_path != '':
    opt.video_path = os.path.join(opt.root_path, opt.video_path)
    opt.annotation_path = os.path.join(opt.root_path, opt.annotation_path)
    opt.result_path = os.path.join(opt.root_path, opt.result_path)
    if opt.resume_path:
        opt.resume_path = os.path.join(opt.root_path, opt.resume_path)
    if opt.pretrain_path:
        opt.pretrain_path = os.path.join(opt.root_path, opt.pretrain_path)
opt.scales = [opt.initial_scale]
for i in range(1, opt.n_scales):
    opt.scales.append(opt.scales[-1] * opt.scale_step)
opt.arch = '{}-{}'.format(opt.model, opt.model_depth)
opt.mean = get_mean(opt.norm_value, dataset=opt.mean_dataset)
opt.std = get_std(opt.norm_value)
print(opt)
with open(os.path.join(opt.result_path, 'opts.json'), 'w') as opt_file:
    json.dump(vars(opt), opt_file)
    

{'no_val': False, 'learning_rate': 0.1, 'n_val_samples': 3, 'scale_step': 0.84089641525, 'std': [38.7568578, 37.88248729, 40.02898126], 'sample_size': 112, 'crop_position_in_test': 'c', 'mean': [114.7748, 107.7354, 99.475], 'result_path': 'results', 'lr_patience': 10, 'pretrain_path': '', 'scale_in_test': 1.0, 'begin_epoch': 1, 'momentum': 0.9, 'n_scales': 5, 'checkpoint': 10, 'train_crop': 'corner', 'scales': [1.0, 0.84089641525, 0.7071067811803005, 0.5946035574934808, 0.4999999999911653], 'wide_resnet_k': 2, 'weight_decay': 0.001, 'no_cuda': False, 'batch_size': 128, 'resnet_shortcut': 'B', 'resnext_cardinality': 32, 'model_depth': 18, 'optimizer': 'sgd', 'arch': 'resnet-18', 'annotation_path': 'kinetics.json', 'no_softmax_in_test': False, 'initial_scale': 1.0, 'nesterov': False, 'no_train': False, 'ft_begin_index': 0, 'n_finetune_classes': 400, 'manual_seed': 1, 'sample_duration': 16, 'dampening': 0.9, 'no_hflip': False, 'mean_dataset': 'activitynet', 'n_epochs': 200, 'dataset': 'ki

In [39]:
torch.manual_seed(opt.manual_seed)

model, parameters = generate_model(opt)
print(model)
criterion = nn.CrossEntropyLoss()
if not opt.no_cuda:
    criterion = criterion.cuda()

if opt.no_mean_norm and not opt.std_norm:
    norm_method = Normalize([0, 0, 0], [1, 1, 1])
elif not opt.std_norm:
    norm_method = Normalize(opt.mean, [1, 1, 1])
else:
    norm_method = Normalize(opt.mean, opt.std)


/home/qq/CSCE689/3D-ResNets-PyTorch/models/resnet.py:145: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')


DataParallel(
  (module): ResNet(
    (conv1): Conv3d(3, 64, kernel_size=(7, 7, 7), stride=(1, 2, 2), padding=(3, 3, 3), bias=False)
    (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
        (bn1): BatchNorm3d(64,

In [40]:

if not opt.no_train:
    assert opt.train_crop in ['random', 'corner', 'center']
    if opt.train_crop == 'random':
        crop_method = MultiScaleRandomCrop(opt.scales, opt.sample_size)
    elif opt.train_crop == 'corner':
        crop_method = MultiScaleCornerCrop(opt.scales, opt.sample_size)
    elif opt.train_crop == 'center':
        crop_method = MultiScaleCornerCrop(
            opt.scales, opt.sample_size, crop_positions=['c'])
    spatial_transform = Compose([
        crop_method,
        RandomHorizontalFlip(),
        ToTensor(opt.norm_value), norm_method
    ])
    temporal_transform = TemporalRandomCrop(opt.sample_duration)
    target_transform = ClassLabel()
    training_data = get_training_set(opt, spatial_transform,
                                     temporal_transform, target_transform)
    train_loader = torch.utils.data.DataLoader(
        training_data,
        batch_size=opt.batch_size,
        shuffle=True,
        num_workers=opt.n_threads,
        pin_memory=True)
    train_logger = Logger(
        os.path.join(opt.result_path, 'train.log'),
        ['epoch', 'loss', 'acc', 'lr'])
    train_batch_logger = Logger(
        os.path.join(opt.result_path, 'train_batch.log'),
        ['epoch', 'batch', 'iter', 'loss', 'acc', 'lr'])

    if opt.nesterov:
        dampening = 0
    else:
        dampening = opt.dampening
    optimizer = optim.SGD(
        parameters,
        lr=opt.learning_rate,
        momentum=opt.momentum,
        dampening=dampening,
        weight_decay=opt.weight_decay,
        nesterov=opt.nesterov)
    scheduler = lr_scheduler.ReduceLROnPlateau(
        optimizer, 'min', patience=opt.lr_patience)



FileNotFoundError: [Errno 2] No such file or directory: 'kinetics.json'

In [ ]:
if not opt.no_val:
    spatial_transform = Compose([
        Scale(opt.sample_size),
        CenterCrop(opt.sample_size),
        ToTensor(opt.norm_value), norm_method
    ])
    temporal_transform = LoopPadding(opt.sample_duration)
    target_transform = ClassLabel()
    validation_data = get_validation_set(
        opt, spatial_transform, temporal_transform, target_transform)
    val_loader = torch.utils.data.DataLoader(
        validation_data,
        batch_size=opt.batch_size,
        shuffle=False,
        num_workers=opt.n_threads,
        pin_memory=True)
    val_logger = Logger(
        os.path.join(opt.result_path, 'val.log'), ['epoch', 'loss', 'acc'])

In [ ]:
if opt.resume_path:
    print('loading checkpoint {}'.format(opt.resume_path))
    checkpoint = torch.load(opt.resume_path)
    assert opt.arch == checkpoint['arch']

    opt.begin_epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['state_dict'])
    if not opt.no_train:
        optimizer.load_state_dict(checkpoint['optimizer'])

In [ ]:
print('run')
for i in range(opt.begin_epoch, opt.n_epochs + 1):
    if not opt.no_train:
        train_epoch(i, train_loader, model, criterion, optimizer, opt,
                    train_logger, train_batch_logger)
    if not opt.no_val:
        validation_loss = val_epoch(i, val_loader, model, criterion, opt,
                                    val_logger)

    if not opt.no_train and not opt.no_val:
        scheduler.step(validation_loss)

In [ ]:
if opt.test:
    spatial_transform = Compose([
        Scale(int(opt.sample_size / opt.scale_in_test)),
        CornerCrop(opt.sample_size, opt.crop_position_in_test),
        ToTensor(opt.norm_value), norm_method
    ])
    temporal_transform = LoopPadding(opt.sample_duration)
    target_transform = VideoID()

    test_data = get_test_set(opt, spatial_transform, temporal_transform,
                             target_transform)
    test_loader = torch.utils.data.DataLoader(
        test_data,
        batch_size=opt.batch_size,
        shuffle=False,
        num_workers=opt.n_threads,
        pin_memory=True)
    test.test(test_loader, model, opt, test_data.class_names)